In [1]:
# Bibliotecas Necessárias
!sudo apt update -y
!sudo apt install -y tesseract-ocr poppler-utils tesseract-ocr-por
!pip install -q pytesseract pdfplumber pdf2image Pillow

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,126 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,267 kB]
Hit:13 https://ppa.launchpadcontent.

In [2]:
# Imports Necessários
import os
from PIL import Image
import pytesseract
import pdfplumber
from pdf2image import convert_from_path

In [3]:
# Funções de Extração

def extrair_texto_pdf(arquivo_pdf):
  """
  Extrai o texto de um arquivo PDF
  """
  texto_extraido = ""
  try:
    with pdfplumber.open(arquivo_pdf) as pdf:
      print(f"Extraindo texto do arquivo: {arquivo_pdf}")
      for pagina in pdf.pages:
        texto_pagina = pagina.extract_text()
        if texto_pagina:
          texto_extraido += texto_pagina + "\n"

    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo do PDF '{arquivo_pdf}")

  except Exception as e:
    print(f"Erro ao extrair texto do PDF: {e}")
    texto_extraido = ""

  return texto_extraido


def aplicar_OCR_imagem(arquivo_imagem):
  """
  Aplica OCR em uma imagem e retorna o texto extraído
  """
  texto_extraido = ""
  try:
    print(f"Aplicando OCR na imagem: {arquivo_imagem}")
    imagem = Image.open(arquivo_imagem)
    texto_extraido = pytesseract.image_to_string(imagem, lang = 'por')

    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo da imagem '{arquivo_imagem}'")

  except Exception as e:
    print(f"Erro ao aplicar OCR na imagem: {arquivo_imagem}: {e}")
    texto_extraido = ""

  return texto_extraido

def aplicar_OCR_pdf(arquivo_pdf):
  """
  Aplica OCR em um PDF só de imagens e retorna o texto extraído
  """
  texto_extraido = ""
  try:
    print(f"Aplicando OCR no PDF: {arquivo_pdf}")
    imagens = convert_from_path(arquivo_pdf, dpi = 300)
    for i, imagem in enumerate(imagens):
      texto_pagina = pytesseract.image_to_string(imagem, lang = 'por')
      if texto_pagina:
        texto_extraido += texto_pagina + "\n"
    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo do PDF '{arquivo_pdf}'")

  except Exception as e:
    print(f"Erro ao aplicar OCR no PDF: {e}")
    texto_extraido = ""
  return texto_extraido

In [4]:
# Função Princuipal

def extrair_texto_de_arquivos(arquivo):
  """
  Extrai o texto de arquivos PDF ou Imagens. Usando OCR se necessário
  """
  texto_extraido = ""
  extencao = os.path.splitext(arquivo)[1].lower()
  print(f"\n--- Processando documento: {arquivo} ---")

  if extencao == ".pdf":
    print("Extraindo texto de arquivo PDF...")
    texto_extraido = extrair_texto_pdf(arquivo)

    if not texto_extraido.strip or len(texto_extraido.strip()) < 20:
      print("Texto extraido do arquivo pequeno ou vazio, tentando aplicar OCR...")
      texto_extraido_ocr = aplicar_OCR_pdf(arquivo)
      if texto_extraido_ocr.strip():
        texto_extraido = texto_extraido_ocr
      else:
        print("Nenhum texto extraido com OCR.")

  elif extencao in [".jpg", ".jpeg", ".png", ".webp"]:
    texto_extraido = aplicar_OCR_imagem(arquivo)

  else:
    print(f"Extensão de arquivo não suportada: {extencao}")

  if not texto_extraido:
    print(f"AVISO FINAL: Nenhum texto significativo extraído de {arquivo}.")

  else:
    print(f"Texto extraido de {arquivo}:")
    print(f"Número de caracteres extraidos: {len(texto_extraido)}")

  return texto_extraido


def salvar_textos(textos_extraidos, saida = 'todo_texto_extraido'):
  os.makedirs(saida, exist_ok = True)
  for arq , texto in textos_extraidos.items():
    nome_original = os.path.basename(arq)
    nome_saida = os.path.join(saida, f"{os.path.splitext(nome_original)[0]}.txt")
    with open(nome_saida, 'w', encoding = 'utf-8') as arquivo:
      arquivo.write(texto)
    print(f"Texto salvo de {nome_original} em: {nome_saida}")

In [5]:
# Prepara os .txts

documentos_oficiais = [
    "/content/documents/CÓDIGO DE OBRAS.pdf",
    "/content/documents/tabela.webp"
]


textos_extraidos = {}

for docs in documentos_oficiais:
  if os.path.exists(docs):
    print(f"Processando documento: {docs}")
    texto_extraido = extrair_texto_de_arquivos(docs)
    if texto_extraido:
      textos_extraidos[docs] = texto_extraido
  else:
    print(f"Documento não encontrado em {docs}. Faça o upload ou confirme o path")


salvar_textos(textos_extraidos)


Processando documento: /content/documents/CÓDIGO DE OBRAS.pdf

--- Processando documento: /content/documents/CÓDIGO DE OBRAS.pdf ---
Extraindo texto de arquivo PDF...
Extraindo texto do arquivo: /content/documents/CÓDIGO DE OBRAS.pdf
Texto extraido de /content/documents/CÓDIGO DE OBRAS.pdf:
Número de caracteres extraidos: 204239
Processando documento: /content/documents/tabela.webp

--- Processando documento: /content/documents/tabela.webp ---
Aplicando OCR na imagem: /content/documents/tabela.webp
Texto extraido de /content/documents/tabela.webp:
Número de caracteres extraidos: 2500
Texto salvo de CÓDIGO DE OBRAS.pdf em: todo_texto_extraido/CÓDIGO DE OBRAS.txt
Texto salvo de tabela.webp em: todo_texto_extraido/tabela.txt


In [ ]:
# Teste Extração de Texto

documentos = [
    "/content/documents/CÓDIGO DE OBRAS.pdf",
    "/content/documents/Desafio Técnico - Dev IA.docx.pdf",
    "/content/documents/Email_DrBergman.png",
    "/content/documents/tabela.webp"
]

textos_extraidos = {}

for documento in documentos:
  if os.path.exists(documento):
    print(f"Processando documento: {documento}")
    texto_extraido = extrair_texto_de_arquivos(documento)
    if texto_extraido:
      textos_extraidos[documento] = texto_extraido
  else:
    print(f"Documento não encontrado em {documento}. Faça o upload ou confirme o path")

print("\n----- Resumo do Texto Extraido -----\n")
for caminho, texto in textos_extraidos.items():
    print(f"\nDocumento: {caminho}")
    print(f"Primeiros 100 caracteres:\n{texto[:100]}...")
    print(f"Total de Caracteres: {len(texto)}")

Processando documento: /content/documents/CÓDIGO DE OBRAS.pdf

--- Processando documento: /content/documents/CÓDIGO DE OBRAS.pdf ---
Extraindo texto de arquivo PDF...
Extraindo texto do arquivo: /content/documents/CÓDIGO DE OBRAS.pdf
Texto extraido de /content/documents/CÓDIGO DE OBRAS.pdf:
Número de caracteres extraidos: 204239
Processando documento: /content/documents/Desafio Técnico - Dev IA.docx.pdf

--- Processando documento: /content/documents/Desafio Técnico - Dev IA.docx.pdf ---
Extraindo texto de arquivo PDF...
Extraindo texto do arquivo: /content/documents/Desafio Técnico - Dev IA.docx.pdf
Texto extraido de /content/documents/Desafio Técnico - Dev IA.docx.pdf:
Número de caracteres extraidos: 1894
Processando documento: /content/documents/Email_DrBergman.png

--- Processando documento: /content/documents/Email_DrBergman.png ---
Aplicando OCR na imagem: /content/documents/Email_DrBergman.png
Texto extraido de /content/documents/Email_DrBergman.png:
Número de caracteres extraido